[データサイエンスのためのPython入門16〜DataFrameのテーブル結合を完全解説\(merge, join, concat\)〜](https://datawokagaku.com/dataframe_merge/)

テーブルの結合をやっていくよ。SQLを理解しているひとには、以下の図がわかりやすいですね。

![](https://datawokagaku.com/wp-content/uploads/2020/02/merge_eyechach.png)

In [1]:
import pandas as pd
import numpy as np

In [15]:
df1 = pd.DataFrame({
    'Key': ['k0', 'k1', 'k2'],
    'A':   ['a0', 'a1', 'a2'],
    'B':   ['b0', 'b1', 'b2'],
})
df2 = pd.DataFrame({
    'Key': ['k0', 'k1', 'k2'],
    'C':   ['c0', 'c1', 'c2'],
    'D':   ['d0', 'd1', 'd2'],
})

In [6]:
df1 

,Key,A,B
0,k0,a0,b0
1,k1,a1,b1
2,k2,a2,b2


In [7]:
df2

,Key,C,D
0,k0,c0,d0
1,k1,c1,d1
2,k2,c2,d2


In [8]:
df1.merge(df2)

,Key,A,B,C,D
0,k0,a0,b0,c0,d0
1,k1,a1,b1,c1,d1
2,k2,a2,b2,c2,d2


`df.merge()`で済む場合にはこれでいいけど、そうじゃない場合もあるので、縦横で結合するやり方をみていこう。

In [10]:
pd.concat([df1, df2], axis=0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,A,B,C,D,Key
0,a0,b0,NaN,NaN,k0
1,a1,b1,NaN,NaN,k1
2,a2,b2,NaN,NaN,k2
0,NaN,NaN,c0,d0,k0
1,NaN,NaN,c1,d1,k1
2,NaN,NaN,c2,d2,k2


当然、NaNになるデータが出てくる。今度は横に結合していく。

In [11]:
pd.concat([df1, df2], axis=1)

,Key,A,B,Key,C,D
0,k0,a0,b0,k0,c0,d0
1,k1,a1,b1,k1,c1,d1
2,k2,a2,b2,k2,c2,d2


`縦につなげる方が圧倒的に多い気がします`とあるのだが、Kaggleでも横に特徴量をつなげていく手法もたくさんあるだろうし、SQL的に行っても横結合は普通にやるし、そんなこともないのではないかなあという気もする。

さっきは行列がそろっているデータ同士でmergeしたから普通にいい感じになったけど、次元が違うデータも普通にある。そういう時に、`df.merge`のオプションを使う。

オプションにはいかがあるよ。

* how : どう結合するか→{‘left’, ‘right’, ‘outer’, ‘inner’}, デフォルトは ‘inner’
* on : keyにするカラムを指定（どちらのDataFrameにも存在するカラム）．指定をしないと共通のカラムで結合される
* left_on：leftのDataFrameのkeyにするカラム
* right_on：rightのDataFrameのkeyにするカラム
* left_index：leftのKeyをindexにする場合Trueを指定
* right_index：rightのKeyをindexにする場合Trueを指定

`how`でleft joinしてみる。

In [20]:
df1 = pd.DataFrame({
    'Key': ['k0', 'k1', 'k2'],
    'A':   ['a0', 'a1', 'a2'],
    'B':   ['b0', 'b1', 'b2'],
})
df2 = pd.DataFrame({
    'Key': ['k0', 'k1', 'k3'],
    'C':   ['c0', 'c1', 'c2'],
    'D':   ['d0', 'd1', 'd2'],
})

この行列をmergeすると、`df1`には`k2`の`C`、`D`がないから、そこの値はNaNになるよ。

In [17]:
df1.merge(df2, how='left')

,Key,A,B,C,D
0,k0,a0,b0,c0,d0
1,k1,a1,b1,c1,d1
2,k2,a2,b2,NaN,NaN


今度は`outer`でやってみるとよ。今度は`k2`の`C`、`D`, `k3`の`A`、`B`がないから、そこがNaNになるよ。

In [18]:
df1.merge(df2, how='outer')

,Key,A,B,C,D
0,k0,a0,b0,c0,d0
1,k1,a1,b1,c1,d1
2,k2,a2,b2,NaN,NaN
3,k3,NaN,NaN,c2,d2


`inner`の場合、`df1`と`df2`に共通してある部分だけとるからNaNはないよ。

In [19]:
df1.merge(df2, how='inner')

,Key,A,B,C,D
0,k0,a0,b0,c0,d0
1,k1,a1,b1,c1,d1


実務で表を結合するときは、大きな表に小さな表を結合していくケースが多いよ。

今度はどのカラムをkeyにして結合するかを指定していくよ。

In [22]:
df1 = pd.DataFrame({
    'Key': ['k0', 'k1', 'k2'],
    'ID':  ['aa', 'bb', 'cc'],
    'A':   ['a0', 'a1', 'a2'],
    'B':   ['b0', 'b1', 'b2'],
})
df2 = pd.DataFrame({
    'Key': ['k0', 'k1', 'k3'],
    'ID':  ['aa', 'bb', 'cc'],
    'C':   ['c0', 'c1', 'c2'],
    'D':   ['d0', 'd1', 'd2'],
})

In [23]:
df1

,Key,ID,A,B
0,k0,aa,a0,b0
1,k1,bb,a1,b1
2,k2,cc,a2,b2


In [24]:
df2 

,Key,ID,C,D
0,k0,aa,c0,d0
1,k1,bb,c1,d1
2,k3,cc,c2,d2


KeyカラムをKeyにして結合するよ。

In [25]:
df1.merge(df2, on='Key')

,Key,ID_x,A,B,ID_y,C,D
0,k0,aa,a0,b0,aa,c0,d0
1,k1,bb,a1,b1,bb,c1,d1


IDからむをKeyにして結合するよ。

In [30]:
df1.merge(df2, on='ID', suffixes=('_left', '_right'))

,Key_left,ID,A,B,Key_right,C,D
0,k0,aa,a0,b0,k0,c0,d0
1,k1,bb,a1,b1,k1,c1,d1
2,k2,cc,a2,b2,k3,c2,d2


結合したいキーの名前が違う場合は、`left_on`, `right_on`をそれぞれ指定するよ。

In [31]:
df1 = pd.DataFrame({
    'Key1': ['k0', 'k1', 'k2'],
    'A':   ['a0', 'a1', 'a2'],
    'B':   ['b0', 'b1', 'b2'],
})
df2 = pd.DataFrame({
    'Key2': ['k0', 'k1', 'k3'],
    'C':   ['c0', 'c1', 'c2'],
    'D':   ['d0', 'd1', 'd2'],
})

In [32]:
df1.merge(df2, left_on='Key1', right_on='Key2')

,Key1,A,B,Key2,C,D
0,k0,a0,b0,k0,c0,d0
1,k1,a1,b1,k1,c1,d1


`df.join()`ってのもあるけど、`merge()`でできるし、使うことないよー、とのこと。